Этот код способен из "приветственных сообщений" доставать ключевые фразы.


К сожалению log с сообщениями и полную версию программы предоставить не могу, так как это конфицииальные данные и не только моя собственность, но для подтверждения показываю самую первую версию программы 

In [26]:
import pandas as pd

data = pd.read_excel("Mesto chat log.xlsx")
data

,Text
0,"Всем привет, я Аня.Живу в Москве.Я практикующи..."
1,Всем привет! Меня зовут Андрей. Образование: б...
2,Привет! Меня зовут Макс и у меня есть нескольк...
3,"Привет гайз. Меня зовут Саша. Круто, что тепер..."
4,"Я и учился в Бонни и слайд, и потом сам руку н..."
...,...
2973,"Всем привет! Меня зовут Денис, родом из Иркутс..."
2974,Всем привет. Я живу в Великобритании и работаю...
2975,Всем привет! Меня зовут Александр. Родился в Т...
2976,"Привет! Я Яна Андреева. Жила в Хабаровске, Нов..."


#### ИМПОРТ ДОПОЛНИТЕЛЬНЫХ МОДУЛЕЙ

In [27]:

import re
from yargy.predicates import (
    eq, in_,
    type, normalized,
    dictionary,
    gte, lte, is_capitalized, gram
)
from yargy import (
    Parser,
    rule, or_, and_, not_
)
from natasha import (
    Segmenter,
    MorphVocab,
     NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    Doc
)
from yargy.tokenizer import Tokenizer

from yargy.pipelines import (
    caseless_pipeline,
    morph_pipeline
)

from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp

####  Функциии  

In [28]:
# функция для поиска нужных сущ.
def finder_noun(doc):
    for i in doc.morph.tokens:
        if unnecessary_word_parser.find(i.text) != None: #не учитываем слова, которые в нашем словаре ненужных слов
            if ('Gender' in i.feats) and (i.pos=='NOUN'):
                if (i.feats['Gender'] == 'Masc') and ('Dat' not in i.feats.values()) and ('Gen' not in i.feats.values() ) :
                    if i.text not in delete_list: 
                        res_doc.append(i.text)
        #                     print(i.text)
                    
                
                
# поиск английских фраз 
def finder_english_words(text):
    try:
        english_phrase = re.finditer(r'(([a-zA-Z][0-9_]|[a-zA-Z])+([\s]+|[!?,.\\\/]))+',text)
        for phrase in english_phrase:
            if ('www.'not in phrase[0]) and ('bit.ly/' not in phrase[0]) and ('.com' not in phrase[0]) and ('.ru' not in phrase[0]) and ('t.me/'not in phrase[0] )and ('Mesto' not in phrase[0]):
                english_word.append(phrase[0].strip(',.!? '))
                for_statistic.append(phrase[0].strip(',.!? '))
#                 print(phrase[0].strip(',.!? '))
    except:
        pass
        
# лемматизация уже отобранных существительных        
def lemmatizee_word_and_add(res_doc,res_doc_lem):
    res_str = ' '.join(res_doc)
    str_doc =  Doc(res_str)
    str_doc.segment(segmenter)
    str_doc.tag_morph(morph_tagger)
    str_doc.parse_syntax(syntax_parser)
    

    for i in str_doc.tokens:
        i.lemmatize(morph_vocab)
        res_doc_lem.append(i.lemma.lower())
        for_statistic.append(i.lemma.lower())
    res_doc_lem =  list(set(res_doc_lem))


In [29]:
# словарь слов, которые не нужны нам 
words_to_del = { 'привет', 'опыт', 'человек', 'вопрос', 'проект', 'продукт','час', 'мир', 'день',
                'момент', 'раз', 'путь','единомышленник','специалист', 'совет',
                'партнер','этап','контракт','ом','друг','навык','результат',
                'год','центр','шаг','штат','конкурс','ребенок','студент',
                'Mesto','edited','бюджет', 'рынок','прототип','смысл','месяц', 'взгляд', 'интерес'}
# сюда будут записываться именованные сущности, которые мы не хотим учитывать 
delete_list = ['времени суток']

In [30]:
# инициализация парсеров

# здесь учитываем все возможные варинаты 
pr= rule(or_(rule(gram('VERB'),gram('NOUN').repeatable()),
            rule(gram('NOUN').repeatable()),
            rule(gram('VERB'),gram('PREP'),
            rule(gram('ADJF').repeatable(), normalized('язык')),
            rule(normalized('делать'), gram('NOUN').repeatable()),
            rule(r'[a-z]+'),
            rule(gram('NOUN').repeatable()))))

# частные случаи    
unnecessary_word_pr = rule(not_(dictionary(words_to_del) ))
verb_pr = rule(rule(gram('NOUN'),gram('NOUN')))
special_pr = rule(gram('ADJF').repeatable(), normalized('язык'))




# запуск парсеров 
parser=Parser(pr)
unnecessary_word_parser = Parser(unnecessary_word_pr)
verb_parser = Parser(verb_pr)
special_parser = Parser(special_pr)

In [31]:
NUMBER_TEXT = 300
res = []  
for_statistic=[] # для подсчета самых популярных слов потом 


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
text_number = -1
for text in data.Text[:NUMBER_TEXT]:
    text_number +=1 
    english_word = []
    res_doc = []
    res_doc_phrase = []
    res_doc_lem = []
    print('-----------------------')
    print(f'{text_number}.  {text}')
    tokenizer = Tokenizer()
    
    try:
        list_of_word = list(tokenizer(text))                         
        for match in parser.findall(text):
            start, stop = match.span
            s = text[start:stop] #наш глагол с сущ.
#             print(text[start:stop])

            doc = Doc(s)
            doc.segment(segmenter)
            doc.tag_morph(morph_tagger)
            doc.parse_syntax(syntax_parser)

            #выявление именованных сущностей 
            try:
                doc.tag_ner(ner_tagger)
    #             print(doc.spans[0].text) 
                delete_list.append(doc.spans[0].text)
            except: 
                pass

            
             # Добавляем болк, где будем учитывать сущ + сущ 
            if verb_parser.find(text[start:stop]):
                if (('Gen' in doc.morph.tokens[1].feats.values())) :
#                     print(f'МЫ НАШЛИ  ---- {text[start:stop]}')
                    if( text[start:stop].split()[0] not in delete_list) and (text[start:stop].split()[1] not in delete_list):
                        res_doc_phrase.append(text[start:stop])
                
                # поиск словосочетаний с языком не работает 
                elif(special_parser.find(text[start:stop])):
#                     print(f'ok ----------{text[start:stop]}')
                    res_doc_phrase.append(text[start:stop])
                        
                else:
                    finder_noun(doc)
                        
            else:
                finder_noun(doc)
    except:
        print('пустой блок без текста')

                
# лемматизация уже отобранных слов 
    lemmatizee_word_and_add(res_doc,res_doc_lem)
# поиск англ фраз
    finder_english_words(text)            
       
    res_doc_lem = res_doc_phrase + res_doc_lem + english_word
    print(res_doc_lem) 
    res.append(res_doc_lem) #добавляем лемматизированные части раздела в массив 
    for_statistic += res_doc_phrase


-----------------------
0.  Всем привет, я Аня.Живу в Москве.Я практикующий карьерный и бизнес коуч.Знаю бизнес-процессы не на словах, а изнутри, за спиной экономическое образование, опыт управленческой должности и магистратура по психологии.На равне с классическими коучинговыми методиками, активно применяю в своей деятельности эмбодименд и майнфулнесс подходы, то есть помогаю людям бизнеса стать эффективнее, работать в команде и управлять ей через внимание к себе, своим движениям и своему телу.Пишу научную работу о внедрении методик mindfulness и embodiment в бизнес коучинг. Приглашаю на бесплатные консультации на темы: повышения эффективности, эмоциональное выгорание, управление и формирование команды, увеличение доходов, смена профессиональной деятельности.Пишите, буду рада сотрудничеству!
['внедрении методик', 'повышения эффективности', 'увеличение доходов', 'бизнес', 'коучий', 'бизнес', 'процесс', 'равний', 'методика', 'подход', 'бизнес', 'mindfulness', 'embodiment']
-------------

['стратегии продвижения в соцсетях', 'фанат изучения', 'интернет', 'маркетингом', 'сайт', 'лендинга', 'менеджер', 'стратег', 'соцсеть', 'таргет', 'фанат', 'клиент', 'клиент', 'отзыв', 'клиент', 'сервис', 'фидбек', 'таргет', 'интернет', 'курс', 'фидбек', 'путешественник', 'пример', 'SMM']
-----------------------
12.  Всем привет!Меня зовут Илья, мне 21.Я юрист в сфере интеллектуальной собственности (авторское право, патенты, товарные знаки) и IT. Осуществляю правовое сопровождение IT-проектов, защиту персональных данных и информации. Имею опыт работы с киберспортивными стриминговыми платформами, зарубежными стартапами, фондами, Big Data, облачными сервисами, товарными знаками и служебными произведениями.Помимо узкой специализации, я и мои коллеги занимаемся правовым сопровождением бизнеса в целом - от создания до ликвидации.Также увлекаюсь живописью, поэзией, eSports, стендапом, английским языком. Отлично, если тут найдутся люди с такими же интересами)Буду рад быть полезным в реализации

['из города Алматы', 'решения проблем в воспитании и развитии', 'поколение детей', 'убеждений прошлого', 'будущего поколения', 'психолог', 'практиковать', 'родитель', 'курс', 'формат', 'аккаунт', 'интернет', 'возраст', 'конверт', 'материал', 'сноуборд']
-----------------------
22.  Всем привет Меня зовут Даша Рыжкова, Product Manager, сейчас базируюсь в Москве в эти странные времена :) Вместе с командой работаем над созданием сервиса для инженерных команд, которые готовятся запустить космический спутник (или несколько). Помогаем им автоматизировать всю рутинную, бумажную работу и работу с supply chain во время их проекта.
['созданием сервиса', 'базироваться', 'спутник', 'Product Manager', 'supply chain']
-----------------------
23.  Привет, я Альберт Родился, вырос и закончил школу в Уфе. После школы поступил в ВУЗ в маленьком студенческом городке Шмалькальден, что находится в Восточной Германии. Учился по специальности информатика. Отучился 3 года и закончил его в сентябре прошлого го

['нашей жизни и развития', 'часть жизни', 'черта характера', 'посредством общения', 'запчасть', 'класс', 'орешек', 'взлет', 'фану', 'родитель', 'майндмап', 'бизнес', 'стресс', 'чат', 'негатив', 'CEO Polean', 'Polean', 'Brain Science', 'AI', 'Polean']
-----------------------
34.  Всем привет! Меня зовут Катя. Сейчас я студентка и у меня еще мало опыта. Но мне всегда очень нравится пробовать себя в новых вещах и знакомиться с новыми людьми. Сейчас живу в Праге.Очень интересуюсь разными сферами в саморазвитии и нейробиологией. Если нужна помощь с иностранными языками(английский,немецкий,чешский,испанский) или какая-нибудь другая,то буду рада помочь! Очень рада всех здесь видеть,отличного дня!:)
['сфера', 'язык']
-----------------------
35.  Всем доброго дня)) Меня зовут Влада, я студентка МГУ, учусь на нейропсихолога, заодно изучаю психотерапию. Не представляю жизни без творчества, люблю облекать свои мысли в формы - хочу обучиться на художника-иллюстратора. Умею учиться и знаю как это де

['пол года жила в Черногории', 'факт', 'процесс', 'инструмент', 'сародиться', 'скалолазание', 'яхтинг', 'made', 'Costa', 'made']
-----------------------
46.  Я Вербицкий Сергей - трабл-шутер.Живу в Киеве.Помогаю найти такое решение в бизнесе, чтобы сложности больше не возвращались.Это как с фокусом. После того как узнаешь секрет фокуса, он перестает быть иллюзией. Владелец бизнеса.Обычно ко мне обращаются со сложными и системными вопросами.Вытащил более 40 бизнесов в кризисы 08 и 14 годов.Веду в ФБ группу «Сам себе трабл-шутер» - https://www.facebook.com/groups/1647583738850786/. Наиболее популярные задачи в 2020 году Управление бизнесом и сообществом. Рост дохода в кармане владельца.Решения сложных бизнес задач.Выход на новые рынки.И получается их решать. J Ментор.Иногда бываю инвестором.Фанатею, когда удается выстроить очередь клиентов у заказчика.И это стало происходить все чаще.Работаю не только в Украине.В нескольких европейских странах.В мае стал работать на рынке Нидерландов. До

...

['комплексы ТВ студий в Лондоне', 'базе сети студий', 'точек присутствия в', 'бизнес', 'город', 'итоге', 'сервис', 'бизнес', 'доступ', 'доступ', 'patron', 'Regus', 'DG6rlsZ1jg_RbkWBBmWStlHUxK_CiBSO/', 'digital video', 'distribution']
-----------------------
266.  Hi everyone! I'm CTO and the only developer as of now of fashion startup Get Outfit (https://getoutfit.ru). I'm Swift/iOS and Flutter/Dart developer and teacher. Former Apple, IBM, Intel. Worked in UK, now based in Moscow, Russia.
['Hi everyone', 'm CTO and the only developer as of now of fashion startup Get Outfit', 'getoutfit', 'm Swift/iOS and Flutter/Dart developer and teacher', 'Former Apple', 'IBM', 'Intel', 'Worked in UK', 'now based in Moscow', 'Russia']
-----------------------
267.  Всем привет! Я Анастасия, мне 34 года, живу в Москве. Прочитала все интервью и в восторге от того, какие все разные и интересные. Рада попасть в такую компанию. Я со-основатель проекта Мобильные игровые 3.02. Мы создаём мобильные игровые с

['кофаундер', 'процесс', 'бизнес', 'паблик', 'спикинг', 'сторителлинг', 'Propertymate', 'Propertymate', 'Techstars Austin', 'Proptech', 'Propertymate', 'Techstars']
-----------------------
273.  Привет! Все здесь такие крутые и классные, что даже немного страшно писать :)Мне 25 лет, я живу в Лондоне 2.5 года и работаю Software Engineer'ом в Facebook. Здесь я занимаюсь разработкой инфраструктуры системы контроля версий исходного кода.До фб я жила в Москве, училась в МГУ и работала в Яндексе в отделе безопасного поиска в команде "Вычисления по IP" (да-да, это не шутки, в трудовой так и записано "по АЙ-ПИ").Главное для меня в работе: решать интересные задачи и профессионально развиваться, ну и самое важное - создавать что-то новое и крутое. Поэтому я готова помогать, чем смогу :)Мои интересы, в основном, это инфраструктура/backend, и есть небольшой опыт Machine Learning.Было бы здорово в сообществе встретить много горящих глаз, крутых идей и желания творить.
['ом', 'отдел', 'Software', 'F

['фандрейзинг ивента', 'вопросам геополитики и', 'стороной бизнеса в любой из', 'опыт создания компании', 'производству мерча', 'мотивам сериалов', 'горизонт', 'университет', 'канал', 'стартап', 'маркетинг', 'банер', 'плакат', 'пост', 'соцсеть', 'куратор', 'директор', 'журнал', 'ом', 'стартап', 'университет', 'текст', 'бизнес', 'выходной', 'язык', 'YouTube', 'cutt.ly/', 'Research', 'mindfulness', 'entertainment', 'Adobe Photoshop', 'Illustrator', 'Indesign', 'Lightroom', 'Premier Pro', 'free']
-----------------------
280.  Господа и дамы, приветствую вас! Я очень рад видеть в одном Месте столько талантливых и открытых людей, глаз радуется!Меня зовут Павел, мне 27 лет, я предприниматель с 17 лет. Основной род деятельности в данный момент это полный цикл  разработки и запуска собственных проектов, в основном мобильных приложений в тематике саморазвития. Например, приложение для медитации - Slowdive.Я человек-оркестр, то есть выполняю всю грязную работу сам - от разработки концепции до ди

['продвижение сайтов', 'ремонту гаджетов', 'дом', 'ресепшионист', 'интернет', 'маркетолог', 'запуск', 'вид', 'муж', 'конце', 'магазин', 'сайт', 'лингвист', 'логист', 'курс', 'скорочтение', 'знакомство', 'Nast', 'dasboards', 'Instagram']
-----------------------
285.  Здравствуйте, друзья!Меня зовут Егор, я живу и работаю в Москве.Я обладаю достаточно глубокой компетенцией:* в девелопменте и управлении проектами в сфере жилой недвижимости* в корпоративных финансах* в инвестиционном анализе комплексных долгосрочных проектов реального сектора* Организации проектного и общего управленияУже 20 лет работаю в сфере девелопмента недвижимости в ведущих российских компаниях: ОПИН, Агранта, Ферро Строй, ПИК, А101, Сафмар.Начинал как специалист казначейства, потом финансовый менеджер, CFO,руководитель проектов в ПИК, Зам. руководителя по девелопменту в А101, а сейчас возглавил направления развития подрядных структур в Сафмар. Был удачно-неудачный опыт своего бизнеса (2 года успешных-на третий год у

['диапазоне температур', 'аттестации материалов', 'фоне', 'случай', 'инженер', 'фильм']
-----------------------
295.  Композитные материалы крутая тема!
['материал']
-----------------------
296.  всем привет! Я фаундер ticketscloud, crypto .tickets, а сейчас фултайм работаю над развитием игры для программистов Geecko.com (edited)
['развитием игры', 'фултайм', 'ticketscloud', 'crypto', 'tickets']
-----------------------
297.  Всем привет, меня зовут АлександрСпасибо за приглашениеЗанимаюсь в основном продуктовым дизайном, начинал свой путь с маркетинг дизайнера, дизайнера коммуникаций (Лендинги, рекламные креативы, все, что поможет привлечь новых клиентов)Мои профили (Конечно, далеко не все портфолио, а только небольшая часть, нужно ими как-то заняться)Behance.com/sater007Dribbble.com/saterОт Места ожидаю новых знакомств, новых крутых проектов и крутого общения с классными людьми (edited)
['дизайнера коммуникаций', 'ожидаю новых знакомств', 'звать', 'дизайн', 'маркетинг', 'креатив', 'кл

#### Самые популярные слова

In [32]:
import collections

c =collections.Counter(for_statistic).most_common()
c

[('стартап', 47),
 ('бизнес', 45),
 ('маркетинг', 29),
 ('сервис', 23),
 ('курс', 16),
 ('IT', 15),
 ('интернет', 15),
 ('запуск', 15),
 ('процесс', 14),
 ('менеджер', 14),
 ('город', 13),
 ('клиент', 12),
 ('веб', 12),
 ('бренд', 11),
 ('канал', 11),
 ('дизайн', 11),
 ('директор', 10),
 ('друг', 10),
 ('предприниматель', 10),
 ('поиск', 9),
 ('основатель', 9),
 ('разработчик', 9),
 ('менеджмент', 8),
 ('навык', 8),
 ('чат', 8),
 ('анализ', 8),
 ('b2b', 8),
 ('сайт', 7),
 ('юрист', 7),
 ('D', 7),
 ('результат', 7),
 ('PR', 7),
 ('аналитик', 7),
 ('программист', 6),
 ('текст', 6),
 ('инженер', 6),
 ('ом', 6),
 ('инвестор', 5),
 ('маркетплейс', 5),
 ('блог', 5),
 ('язык', 5),
 ('профиль', 5),
 ('контент', 5),
 ('продакт', 5),
 ('план', 5),
 ('кофаундер', 5),
 ('магазин', 5),
 ('организатор', 5),
 ('дизайнер', 5),
 ('пользователь', 5),
 ('подход', 4),
 ('ивента', 4),
 ('контакт', 4),
 ('Google', 4),
 ('фонд', 4),
 ('коллега', 4),
 ('банк', 4),
 ('актив', 4),
 ('AI', 4),
 ('инструмент', 4)